In [0]:
%python
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Data").getOrCreate()

In [0]:
%python
data = [
    ("Alice", "HR", 50000, "New York"),
    ("Bob", "IT", 60000, "San Francisco"),
    ("Charlie", "HR", 52000, "New York"),
    ("David", "Finance", 58000, "Chicago"),
    ("Eve", "IT", 61000, "San Francisco"),
    ("Frank", "Marketing", 47000, "Boston"),
    ("Grace", "Finance", 59000, "Chicago"),
    ("Heidi", "HR", 53000, "New York"),
    ("Ivan", "Marketing", 48000, "Boston"),
    ("Judy", "IT", 62000, "San Francisco")
]
columns = ["Name", "Department", "Salary", "Location"]

df=spark.createDataFrame(data,columns)

display(df)

In [0]:
%python
df.write.format("delta").mode("overwrite").saveAsTable("employeedata")

In [0]:
%sql
select * from employeedata

In [0]:
%sql
describe history employeedata


In [0]:
update employeedata set salary=100000 where name='Bob'

In [0]:
describe history employeedata


In [0]:
%python

#MERGE INTO - new row


from pyspark.sql import Row

update_data = [Row(Name="Ayush",Department="Finance",Salary=95000,Location="Pune")]
df_update= spark.createDataFrame(update_data)
df_update.createOrReplaceTempView("updates_view")

spark.sql("""
MERGE INTO employeedata AS target
USING updates_view AS SOURCE
on target.Name = source.Name
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
""")

In [0]:
select * from employeedata

In [0]:
describe history employeedata

In [0]:
%python
#SCHEMA EVALUATION - add new column

new_data = ["David", "HR", 78000, 6.2]
cols_new = ["Name","Department","Salary","Experience"]
df_new=spark.createDataFrame([new_data],cols_new)
display(df_new)

In [0]:
%python
# MERGE SCHEMA  

df_new.write.format("delta") \
    .option("mergeSchema", "true") \
    .mode("append") \
    .saveAsTable("employeedata")

In [0]:
select * from employeedata

In [0]:
describe extended employeedata